In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn as sk

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 1000, max_leaf_nodes = 32, n_jobs = -1, max_depth = 6)

from sklearn.linear_model import LogisticRegression
model2 = LogisticRegression()

from sklearn.svm import LinearSVC
model3 = LinearSVC()

from sklearn.svm import SVC
model4 = SVC(kernel = 'linear', C = 0.025)

from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

#データの読み込み
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [2]:
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:

train = train.drop(['Name', 'Embarked', 'Ticket'], axis = 'columns')
train = train.fillna(train['Age'].median())
train['Sex'] = (train['Sex'] == 'female') * 1
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
train = train.drop(['SibSp', 'Parch'], axis ='columns')
train['Cabin'] = train['Cabin'].apply(lambda x:0 if type(x) == float else 1)


In [14]:
train.info()
train.head()

train_data = train.values
xs = train_data[:, 2:]
y = train_data[:, 1]

print (xs.shape)
print (y.shape)

grid_forest = RandomForestClassifier()
grid_ada = AdaBoostClassifier()
grid_LGBM = LGBMClassifier()
param_grid = {'max_depth' : [2, 3, 4], 'learning_rate' : [0.225, 0.25, 0.275], 'num_leaves' : [3, 4, 5, 6]}
grid_search = GridSearchCV(grid_LGBM, param_grid, cv =3, scoring = "accuracy")
grid_search.fit(xs, y)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null int64
Age            891 non-null float64
Fare           891 non-null float64
Cabin          891 non-null int64
FamilySize     891 non-null int64
dtypes: float64(2), int64(6)
memory usage: 55.8 KB
(891, 6)
(891,)


GridSearchCV(cv=3, error_score=nan,
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=100,
                                      n_jobs=-1, num_leaves=31, objective=None,
                                      random_state=None, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'learning_rate': [0.225, 0.25, 0.275],
                         'max_

In [15]:
grid_search.best_params_

{'learning_rate': 0.25, 'max_depth': 2, 'num_leaves': 4}

In [16]:
#モデルの作成
model = model.fit(xs, y)
model2 = model2.fit(xs, y)
model3 = model3.fit(xs, y)
model4 = model4.fit(xs, y)
model5 = VotingClassifier(estimators=[('rf', RandomForestClassifier(n_estimators = 1000, max_leaf_nodes = 32, n_jobs = -1)),('logi', LogisticRegression())])
model5 = model5.fit(xs, y)
model6 = LGBMClassifier(learning_rate=0.25,max_depth=2,num_leaves=4).fit(xs,y)
model7 = AdaBoostClassifier(n_estimators = 500, learning_rate = 0.008)
model7 = model7.fit(xs, y)
model8 = VotingClassifier(estimators=[('rf', RandomForestClassifier(n_estimators = 5000,\
                                                                    max_leaf_nodes = 16, n_jobs = -1)),\
                                      ('logi', LogisticRegression()),\
                                      ('lgb', LGBMClassifier(learning_rate=0.01,max_depth=2,num_leaves=3))])
model8 = model8.fit(xs, y)
model9 = KNeighborsClassifier(n_neighbors = 3)
model9 = model9.fit(xs, y)
model10 = GaussianNB()
model10 = model10.fit(xs, y)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [17]:
#テストデータにもトレーニングデータと同じ処理を行う
test = test.drop(['Name', 'Embarked', 'Ticket'], axis = 'columns')
test = test.fillna(train['Age'].median())
test['Sex'] = (test['Sex'] == 'female') * 1

test['FamilySize'] = test['SibSp'] + test['Parch'] + 1
test = test.drop(['SibSp', 'Parch'], axis ='columns')

test['Cabin'] = test['Cabin'].apply(lambda x:0 if type(x) == float else 1)


test_data = test.values
xs_test = test_data[:, 1:]

print(xs_test.shape)

(418, 6)


In [18]:
output = model.predict(xs_test).astype(int)
output2 = model2.predict(xs_test).astype(int)
output3 = model3.predict(xs_test).astype(int)
output4 = model4.predict(xs_test).astype(int)
output5 = model5.predict(xs_test).astype(int)
output6 = model6.predict(xs_test).astype(int)
output7 = model7.predict(xs_test).astype(int)
output8 = model8.predict(xs_test).astype(int)
output9 = model9.predict(xs_test).astype(int)
output10 = model10.predict(xs_test).astype(int)

In [19]:
test['Survived'] = output6
test.to_csv('output.csv', columns = ['PassengerId', 'Survived'], index = False)

In [20]:
print(model.feature_importances_)
print(model6.feature_importances_)
print(model7.feature_importances_)

[0.10894665 0.4372347  0.13137598 0.16701952 0.0734486  0.08197456]
[ 12  26  80 129   9  24]
[0.12  0.418 0.082 0.13  0.112 0.138]


In [21]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, xs, y, cv=5, scoring = 'accuracy')
scores2 = cross_val_score(model2, xs, y, cv=5, scoring = 'accuracy')
scores3 = cross_val_score(model3, xs, y, cv=5, scoring = 'accuracy')
scores4 = cross_val_score(model4, xs, y, cv=5, scoring = 'accuracy')
scores5 = cross_val_score(model5, xs, y, cv=5, scoring = 'accuracy')
scores6 = cross_val_score(model6, xs, y, cv=5, scoring = 'accuracy')
scores7 = cross_val_score(model7, xs, y, cv=5, scoring = 'accuracy')
scores8 = cross_val_score(model8, xs, y, cv=5, scoring = 'accuracy')
scores9 = cross_val_score(model9, xs, y, cv=5, scoring = 'accuracy')
scores10 = cross_val_score(model10, xs, y, cv=5, scoring = 'accuracy')

print("Random Forest:        ", scores,"Mean:",  scores.mean())
print("Logistic Regression:  ", scores2,"Mean:",  scores2.mean())
print("LinearSVC:            ", scores3,"Mean:",  scores3.mean())
print("SVC:                  ", scores4,"Mean:",  scores4.mean())
print("Voting Classifier:    ", scores5,"Mean:",  scores5.mean())
print("LightGBM:             ", scores6,"Mean:",  scores6.mean())
print("AdaBoost:             ", scores7,"Mean:",  scores7.mean())
print("New Voting Classifier:", scores8,"Mean:",  scores8.mean())
print("KNeighbors:           ", scores9,"Mean:",  scores9.mean())
print("Goussian NB:          ", scores10,"Mean:",  scores10.mean())

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Library/Fra

Random Forest:         [0.79329609 0.80337079 0.83146067 0.80898876 0.83707865] Mean: 0.814838993157994
Logistic Regression:   [0.80446927 0.80337079 0.78651685 0.78651685 0.82022472] Mean: 0.8002196974452327
LinearSVC:             [0.72625698 0.71348315 0.73033708 0.78651685 0.73595506] Mean: 0.7385098236143368
SVC:                   [0.79888268 0.8258427  0.78089888 0.75280899 0.78651685] Mean: 0.7889900194589166
Voting Classifier:     [0.77653631 0.80898876 0.83146067 0.80898876 0.80898876] Mean: 0.8069926558282594
LightGBM:              [0.79329609 0.83707865 0.85393258 0.81460674 0.85393258] Mean: 0.8305693302366455
AdaBoost:              [0.81005587 0.82022472 0.80337079 0.79213483 0.80337079] Mean: 0.8058313979034587
New Voting Classifier: [0.76536313 0.80898876 0.80898876 0.7752809  0.8258427 ] Mean: 0.7968928504174252
KNeighbors:            [0.66480447 0.68539326 0.7247191  0.74719101 0.71910112] Mean: 0.7082417927311531
Goussian NB:           [0.72067039 0.74157303 0.7752809 

In [22]:
xs = train_data[:, 2:6]
print(xs)
print(xs.shape)
print(y.shape)

[[ 3.      0.     22.      7.25  ]
 [ 1.      1.     38.     71.2833]
 [ 3.      1.     26.      7.925 ]
 ...
 [ 3.      1.     28.     23.45  ]
 [ 1.      0.     26.     30.    ]
 [ 3.      0.     32.      7.75  ]]
(891, 4)
(891,)


In [13]:
model = model.fit(xs, y)
model2 = model2.fit(xs, y)
model3 = model3.fit(xs, y)
model4 = model4.fit(xs, y)
model5 = VotingClassifier(estimators=[('rf', RandomForestClassifier(n_estimators = 1000, max_leaf_nodes = 32, n_jobs = -1)),('logi', LogisticRegression()), ('svc', SVC(kernel = 'linear', C = 0.025))])
model5 = model5.fit(xs, y)
model6 = model6.fit(xs, y)
model7 = model7.fit(xs, y)
model8 = model8.fit(xs, y)
model9 = model9.fit(xs, y)
model10 = model10.fit(xs, y)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [14]:
test_data = test.values
xs_test = test_data[:, 1:5]

print(xs_test.shape)


(418, 4)


In [15]:
output = model.predict(xs_test).astype(int)
output2 = model2.predict(xs_test).astype(int)
output3 = model3.predict(xs_test).astype(int)
output4 = model4.predict(xs_test).astype(int)
output5 = model5.predict(xs_test).astype(int)
output6 = model6.predict(xs_test).astype(int)
output7 = model7.predict(xs_test).astype(int)
output8 = model8.predict(xs_test).astype(int)
output9 = model9.predict(xs_test).astype(int)
output10 = model10.predict(xs_test).astype(int)

In [21]:
test['Survived'] = output
test.to_csv('output.csv', columns = ['PassengerId', 'Survived'], index = False)

In [17]:
print(model.score(xs, y))
print(model2.score(xs, y))
print(model3.score(xs, y))
print(model4.score(xs, y))
print(model5.score(xs, y))
print(model6.score(xs, y))
print(model7.score(xs, y))
print(model8.score(xs, y))
print(model9.score(xs, y))
print(model10.score(xs, y))

0.8720538720538721
0.7991021324354658
0.6374859708193041
0.7867564534231201
0.8047138047138047
0.7867564534231201
0.819304152637486
0.8428731762065096
0.8406285072951739
0.7777777777777778


In [18]:
#交差検証
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, xs, y, cv=5, scoring = 'accuracy')
scores2 = cross_val_score(model2, xs, y, cv=5, scoring = 'accuracy')
scores3 = cross_val_score(model3, xs, y, cv=5, scoring = 'accuracy')
scores4 = cross_val_score(model4, xs, y, cv=5, scoring = 'accuracy')
scores5 = cross_val_score(model5, xs, y, cv=5, scoring = 'accuracy')
scores6 = cross_val_score(model6, xs, y, cv=5, scoring = 'accuracy')
scores7 = cross_val_score(model7, xs, y, cv=5, scoring = 'accuracy')
scores8 = cross_val_score(model8, xs, y, cv=5, scoring = 'accuracy')
scores9 = cross_val_score(model9, xs, y, cv=5, scoring = 'accuracy')
scores10 = cross_val_score(model10, xs, y, cv=5, scoring = 'accuracy')

def display_scores(scores):
    print('scores:', scores)
    print('Mean:', scores.mean())
    print('Standard devitation:', scores.std())



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Library/Fra

In [19]:
print("Random Forest:        ", scores,"Mean:",  scores.mean())
print("Logistic Regression:  ", scores2,"Mean:",  scores2.mean())
print("LinearSVC:            ", scores3,"Mean:",  scores3.mean())
print("SVC:                  ", scores4,"Mean:",  scores4.mean())
print("Voting Classifier:    ", scores5,"Mean:",  scores5.mean())
print("LightGBM:             ", scores6,"Mean:",  scores6.mean())
print("AdaBoost:             ", scores7,"Mean:",  scores7.mean())
print("New Voting Classifier:", scores8,"Mean:",  scores8.mean())
print("New Voting Classifier:", scores9,"Mean:",  scores9.mean())
print("New Voting Classifier:", scores10,"Mean:",  scores10.mean())


Random Forest:         [0.82122905 0.83146067 0.8258427  0.80337079 0.85393258] Mean: 0.8271671583704727
Logistic Regression:   [0.7877095  0.78089888 0.78651685 0.7752809  0.80337079] Mean: 0.7867553825874082
LinearSVC:             [0.75418994 0.73033708 0.74719101 0.74157303 0.79775281] Mean: 0.7542087753436696
SVC:                   [0.79329609 0.80337079 0.78651685 0.75280899 0.78651685] Mean: 0.7845019145063085
Voting Classifier:     [0.79888268 0.82022472 0.78651685 0.76966292 0.80337079] Mean: 0.7957315924926245
LightGBM:              [0.74860335 0.79213483 0.78651685 0.75280899 0.80337079] Mean: 0.7766869625258929
AdaBoost:              [0.77653631 0.80898876 0.78651685 0.79213483 0.8258427 ] Mean: 0.7980038917833155
New Voting Classifier: [0.80446927 0.8258427  0.79213483 0.76966292 0.82022472] Mean: 0.8024668884564686
New Voting Classifier: [0.65363128 0.69662921 0.73595506 0.75280899 0.70224719] Mean: 0.7082543468708806
New Voting Classifier: [0.70949721 0.79775281 0.7808988

In [20]:
print(model.feature_importances_)
print(model6.feature_importances_)
print(model7.feature_importances_)

[0.15066083 0.45942345 0.16449372 0.22542201]
[ 82 100  16   2]
[0.15  0.084 0.29  0.476]
